<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Configuring-panadas" data-toc-modified-id="Configuring-panadas-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Configuring panadas</a></span></li><li><span><a href="#Data-Indexing" data-toc-modified-id="Data-Indexing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Indexing</a></span></li><li><span><a href="#The-importance-of-indexes" data-toc-modified-id="The-importance-of-indexes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>The importance of indexes</a></span></li><li><span><a href="#인덱스-유형" data-toc-modified-id="인덱스-유형-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>인덱스 유형</a></span><ul class="toc-item"><li><span><a href="#The-fundamental-index-type:-Index" data-toc-modified-id="The-fundamental-index-type:-Index-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>The fundamental index type: Index</a></span></li><li><span><a href="#Integer-index-labels-using-Int64Index-and-RangeIndex" data-toc-modified-id="Integer-index-labels-using-Int64Index-and-RangeIndex-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Integer index labels using Int64Index and RangeIndex</a></span></li><li><span><a href="#Floating-point-labels-using-Float64Index" data-toc-modified-id="Floating-point-labels-using-Float64Index-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Floating point labels using Float64Index</a></span></li><li><span><a href="#Representing-discrete-intervals-using-IntervalIndex" data-toc-modified-id="Representing-discrete-intervals-using-IntervalIndex-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Representing discrete intervals using IntervalIndex</a></span></li><li><span><a href="#Categorical-values-as-an-index:-CategoricalIndex" data-toc-modified-id="Categorical-values-as-an-index:-CategoricalIndex-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Categorical values as an index: CategoricalIndex</a></span></li></ul></li></ul></div>

# Configuring panadas
라이브러리 임포트, 버전 확인, 파일 업로드

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd

# used for dates
import datetime
from datetime import datetime, date

# Set some pandas options controlling output format
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 60)

# bring in matplotlib for graphics
import matplotlib.pyplot as plt
%matplotlib inline

# version check
print(pd.__version__)
print(np.__version__)
import sys
print(sys.version)

# read in the data and print the first five rows
# use the Symbol column as the index, and 
# only read in columns in positions 0, 2, 3, 7
sp500 = pd.read_csv("/Users/Angela/Desktop/Personal/Learning-Pandas-Second-Edition-master/data/sp500.csv", 
                    index_col='Symbol', 
                    usecols=[0, 2, 3, 7])

1.4.1
1.22.4
3.8.5 (default, Sep  4 2020, 02:22:02) 
[Clang 10.0.0 ]


# Data Indexing

데이터 인덱싱

인덱스는 시리즈나 데이터 프레임으로부터 값을 검색하기 위한 최적화된 도구로, 관계형 데이터베이스에서의 키와는 다르지만 오히려 더 강력하다. 인덱스는 복수의 데이터셋에 대한 정렬 기능을 제공하며, 서로 다른 빈도의 재표집(resampling)과 같은 다양한 데이터 작업에 의미를 부여하는 역할을 한다.  

pandas로 수행하는 모델링의 상당 부분은 인덱스를 어떻게 설정하느냐에 매우 의존적이다. 제대로 구축한 인덱스는 성능 최적화에 도움이 되며, 데이터 분석에 있어서 매우 가치있는 도구가 된다.  

- 인덱스의 중요성
- pandas의 인덱스 유형 : `RangeIndex, Int64Index, CategoricalIndex, Float64Index, Datetimeindex, PeriodIndex`
- 인덱스 설정과 재설정
- 계층형 인덱스의 생성
- 계층형 인덱스를 사용한 데이터 선택  

# The importance of indexes
인덱스의 중요성

판다스에서 인덱스는 갑스이 효율적인 검색을 가능하게 한다. 인덱스가 없다면 모든 데이터는 선형 검색(linear search, 순차적으로 모두 검색)을 해야한다.  
인덱스는 특정 데이터 아이템에 대한 최적화된 단축키며, 순차적인 과정 없이 한 번에 검색을 수행한다.  

난수 10,00 개를 만들어 실습을 해보도록 한다. key라는 칼럼은 100부터 1씩 증가하는 정수 range를 입력하도록 했다. 

In [2]:
# create DataFame of random numbers and a key column
np.random.seed(123456)
df = pd.DataFrame({'foo':np.random.random(10000), 'key':range(100, 10100)})
df[:5]

        foo  key
0  0.126970  100
1  0.966718  101
2  0.260476  102
3  0.897237  103
4  0.376750  104

key = 10099 인 난수 데이터를 찾으려고 하는데, 이는 DataFrame의 마지막 로우이다. 불리언 선택으로 검색해본다.  
간단한 작업이지만 꽤 오래걸림을 증명해볼 수 있다. 

`%timeit` 구문을 사용해, 반복적으로 검색하는 시뮬레이션을 통해 이 작업의 성능을 테스트 해본다. 

In [3]:
# boolean select where key is 10099
df[df.key==10099]

           foo    key
9999  0.272283  10099

In [4]:
# time the select
%timeit df[df.key==10099]

568 µs ± 174 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


1000 번의 루프를 7번 실행했고, 평균적으로 379나노퍼세크 정도 걸린다고 한다.  

인덱스를 활용해 직접 값을 검색할 것인데, 우리가 찾으려는 값이 있는 key라는 칼럼을 index로 설정한 DataFrame에서 수행한다. 

In [5]:
# move key to the index
df_with_index = df.set_index(['key'])
df_with_index[:5]

          foo
key          
100  0.126970
101  0.966718
102  0.260476
103  0.897237
104  0.376750

`.loc[]`에 우리가 찾으려는 값을 넣어 찾을 수 있다.  
이를 시간을 측정해본다. 

In [6]:
# now can lookup with the index
df_with_index.loc[10099]

foo    0.272283
Name: 10099, dtype: float64

In [7]:
# and this is a lot faster
%timeit df_with_index.loc[10099]

86.3 µs ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


$ \pm $ 까지 계산하면 대략 5배 이상 빠르다. 이러한 성능상의 이유로 가능하다면 인덱스로 검색하는 것이 가장 좋다.  

**인덱스를 사용하는 경우의 단점은 인덱스를 구성할 시간이 필요하며, 메모리를 좀 더 사용한다는 점**이다.  

대부분의 경우 어떤 것이 인덱스가 돼야 하는지 본능적으로 느낄 수 있을 것이며, 따라서 인덱스를 먼저 만들고 데이터 작업을 시작하면 된다. (중복이 없고 숫자처럼 비트가 작을 수록 좋을 것같다).  
그렇지 않을 경우에는 최선의 인덱스를 결정하기 위해 약간의 탐색을 먼저 해봐야 할 것이다. 때로는 충분한 데이터를 갖고 있지 않거나, 인덱스로 만들 적당한 필드가 없을 경우도 있다. 그런 경우 다소 모호한 복수의 결과를 반환하는 **부분적인 인덱스**를 사용할 수도 있는데, 그래도 불리언 선택을 통해 원하는 결과를 볼 수 있다. (중복인 경우 둘 다 반환하는 것을 이전에 확인했음.)  

    탐색적 데이터 분석을 수행할 때에는 먼저 데이터를 로딩하고 쿼리와 불리언 선택을 사용해 탐색하는 방법이 권장된다. 그다음, 자연스럽게 인덱스를 지원할 수 있는 데이터이거나 또는 성능 향상이 필요한 경우에 인덱스를 만들면 된다.  

# 인덱스 유형
---
pandas는 다수의 내장 인덱스를 제공하는데, 각 인덱스 유형은 특정 데이터 타입이나 데이터의 패턴에 기초해 최적의 검색을 위해 설계됐다.  

## The fundamental index type: Index
기본 유형 : `Index`

Index는 가장 포괄적인 인덱스이며, 순서가 있고, 슬라이싱이 가능한 값들의 집합이다.  
인덱스 자체는 반드시 해싱 가능한(hashable) 파이썬 객체여야 한다. 이는 인덱스가 그 해시를 이용해 값 검색을 효율적으로 하기 때문이다. 해시 검색이 선형 검색보다 선호되는 방식이지만, 더욱 최적화 할 수 있는 다른 인덱스 유형들도 있다고.  

보통은 **칼럼 인덱스에 `Index`유형이 사용된다.**  
`Index`는 대개 영숫자 칼럼명에 잘 어울리지만, 원한다면 칼럼 인덱스에 다른 유형을 사용하는 일도 있다. 

In [8]:
# show that the columns are actually an index
temps = pd.DataFrame({ "City": ["Missoula", "Philadelphia"],
                       "Temperature": [70, 80] })
temps

           City  Temperature
0      Missoula           70
1  Philadelphia           80

In [9]:
# we can see columns is an index
temps.columns

Index(['City', 'Temperature'], dtype='object')

## Integer index labels using Int64Index and RangeIndex
정수 유형: `Int64Index`, `RangeIndex`

`Int64Index`는 값과 매핑되는 64비트 정수의 불변 배열(immutable array)을 나타낸다. (얼마 전까지, 인덱스를 명시적으로 지정하지 않았거나 인덱스를 정수로 사용할 때 적용되는 pandas의 기본 인덱스 유형이었다고.) 

이 인덱스 유형은 연속적인 in-memory 배열을 사용하므로써, DataFrame의 로우 검색에 있어서 높은 성능을 보여준다.  

In [10]:
# explicitly create an Int64Index
df_i64 = pd.DataFrame(np.arange(10, 20), index=np.arange(0, 10))
df_i64[:5]

    0
0  10
1  11
2  12
3  13
4  14

In [11]:
# view the index
df_i64.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

 `RangeIndex`는 `Int64Index`를 더욱 최적화한 버전이다. 시작 값, 마지막 값, 그리고 각 값의 간격을 나타낼 수 있는 정수 기반의 인덱스다.  
 시작, 끝, 간격을 이용하는 것은 하위 인덱스 추가를 보장하는 일반적인 패턴이다. 이 세 값을 사용하면 메모리를 절약할 수 있으며, 실행 시간도 `Int64Index`와 동일하다.  
 
 `RangeIndex`는 pandas 객체를 위한 기본 인덱스 유형이 됐다. 

In [12]:
# by default we are given a RangeIndex
df_range = pd.DataFrame(np.arange(10, 15))
df_range[:5]

    0
0  10
1  11
2  12
3  13
4  14

In [13]:
df_range.index

RangeIndex(start=0, stop=5, step=1)

## Floating point labels using Float64Index
부동소수점 유형 : `Float64Index`

부동소수점 수도 `Float64Index`를 통해 인덱스 레이블에 사용될 수 있다. 물론 `.iloc[]`를 사용할 때는 정수를 입력하여 찾아야 한다. 

In [14]:
# indexes using a Float64Index
df_f64 = pd.DataFrame(np.arange(0, 1000, 5), 
                      np.arange(0.0, 100.0, 0.5))
df_f64.iloc[:5] # need iloc to slice first five

      0
0.0   0
0.5   5
1.0  10
1.5  15
2.0  20

In [15]:
df_f64.index

Float64Index([ 0.0,  0.5,  1.0,  1.5,  2.0,  2.5,  3.0,
               3.5,  4.0,  4.5,
              ...
              95.0, 95.5, 96.0, 96.5, 97.0, 97.5, 98.0,
              98.5, 99.0, 99.5],
             dtype='float64', length=200)

## Representing discrete intervals using IntervalIndex
이산 간격 유형 : `IntervalIndex`

명확한 **간격**으로 구분되는 레이블일 경우 `IntervalIndex`를 사용해 나타낼 수 있다.  
이산 간격(discrete interval)은 왼쪽이나 오른쪽 끝에 닫혀 있는데, 이는 그 끝에 있는 인덱스가 간격에 포함됨을 의미한다.  

다음은 `IntervalIndex`를 인덱스로 사용하는 DataFrame을 만드는 예시이다. 

In [16]:
# a DataFrame with an IntervalIndex
df_interval = pd.DataFrame({ "A": [1, 2, 3, 4]},
                    index = pd.IntervalIndex.from_breaks(
                        [0, 0.5, 1.0, 1.5, 2.0]))
df_interval

            A
(0.0, 0.5]  1
(0.5, 1.0]  2
(1.0, 1.5]  3
(1.5, 2.0]  4

In [17]:
df_interval.iloc[:2]

            A
(0.0, 0.5]  1
(0.5, 1.0]  2

In [18]:
df_interval.index

IntervalIndex([(0.0, 0.5], (0.5, 1.0], (1.0, 1.5], (1.5, 2.0]], dtype='interval[float64, right]')

## Categorical values as an index: CategoricalIndex
범주형 : `CategoricalIndex`

`CategoricalIndex`는 범주에 포함되지만 흩어져 있는 인덱스를 나타낸다.  

다음은 하나의 칼럼을 범주로 지정하여 DataFrame을 만드는 방식이다. 